<H2>Print attribute values for checking</H2>

In [2]:
import pandas as pd
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.22.0")
if not lusas.existsDatabase():
    Exception("A model must be open before running this code")

db = lusas.database() # Reference to the database for convenience

# 1. Write out attribute values

In [3]:
# Check if the geometic attribute with ID of 1 exists
if lusas.db().existsAttribute("Geometric", 1):
    # Get the geometric attribute with ID=1
    geom_attr = lusas.db().getAttribute("Geometric", 1)

    # Loop through the attributes value names and print out the corresponding values and units
    for name in geom_attr.getValueNames():
        print(f"{name} : {geom_attr.getValue(name)} {geom_attr.getValueUnits(name)}")

Or we can request them in a particular unit set

In [4]:
# Check if the geometic attribute with ID of 1 exists
if lusas.db().existsAttribute("Geometric", 1):
    # Get the geometric attribute with ID=1
    geom_attr = lusas.db().getAttribute("Geometric", 1)

    # Determine a unit set
    si_units = lusas.getUnitSet("N,mm,t,s,C")

    # Loop through the attributes value names and print out the corresponding values and units in the given unit set
    for name in geom_attr.getValueNames():
        print(f"{name} : {geom_attr.getValue(name, si_units)} {geom_attr.getValueUnits(name, si_units)}")

Generate a table of all attribute values and export to excel

In [5]:
# Get all the geometric attributes
all_geometric_attributes = lusas.db().getAttributes("Geometric")

# 2. Combination definitions

2.1 Basic Combinations

In [6]:
# Get all the basic combinations
all_basic_combinations = lusas.db().getLoadsets("Basic Combinations")

# Create a list to store all the combination data
combs_data = []
# Maintain the max number of loadsets in the combinations so we can create the correct number of column headers
max_loadsets = 0

for combination in all_basic_combinations:
    # Get a list of all the loadsets in the ocmbination
    ids = combination.getLoadcaseIDs()
    # Get a list of all the corresponding factors
    factors = combination.getFactors()
    # Create a list which will store all loadset names and factors but starts with the combination name
    combination_loadsets_and_factors = [combination.getIDAndName()]
    # Append each loadset name followed by the factor used
    for i, id in enumerate(ids):
        loadset = lusas.db().getLoadset(id)
        combination_loadsets_and_factors.append(loadset.getIDAndNameAndDescription())
        combination_loadsets_and_factors.append(factors[i])

    max_loadsets = max(max_loadsets, len(ids))
    
    combs_data.append(combination_loadsets_and_factors)

# Create a list for the column headers
column_headers = ["Combination"] + ["Name", "Factor"]*max_loadsets
# Use a pandas dataframe to export to xlsx format
df = pd.DataFrame(combs_data, columns= column_headers)

import os
desktop = os.path.expanduser("~/Desktop")
df.to_excel(rf"{desktop}\{lusas.db().getDBBasename()}-Basic Combinations.xlsx")
# Print the top few elements of the dataframe as a check
print(df.head())

Empty DataFrame
Columns: [Combination]
Index: []


2.2 Smart combinations

In [ ]:
# Get all the smart combinations
all_smart_combinations = lusas.db().getLoadsets("Smart Combinations")

# Create a list to store all the combination data
combs_data = []
# Maintain the max number of loadsets in the combinations so we can create the correct number of column headers
max_loadsets = 0

for combination in all_smart_combinations:
    # Smart combinations have two loadsets but they have equal definitions so we'll just report the max case
    if combination.isMax():
        # Get a list of all the loadsets in the ocmbination
        ids = combination.getLoadcaseIDs()
        # Get a list of all the corresponding permanent factors
        permanent_factors = combination.getPermanentFactors()
        # Get a list of all the corresponding variable factors
        variable_factors = combination.getVariableFactors()
        # Create a list which will store all loadset names and factors but starts with the combination name
        combination_loadsets_and_factors = [combination.getIDAndName()]
        # Append each loadset name followed by the factor used
        for i, id in enumerate(ids):
            loadset = lusas.db().getLoadset(id)
            combination_loadsets_and_factors.append(loadset.getIDAndNameAndDescription())
            combination_loadsets_and_factors.append(permanent_factors[i])
            combination_loadsets_and_factors.append(permanent_factors[i]+variable_factors[i]) # Note that the adverse factor is made up of a permanent and variable parts

        max_loadsets = max(max_loadsets, len(ids))
        
        combs_data.append(combination_loadsets_and_factors)

# Create a list for the column headers
column_headers = ["Combination"] + ["Name", "Beneficial", "Adverse"]*max_loadsets
# Use a pandas dataframe to export to xlsx format
df = pd.DataFrame(combs_data, columns= column_headers)

import os
desktop = os.path.expanduser("~/Desktop")
df.to_excel(rf"{desktop}\{lusas.db().getDBBasename()}-Smart Combinations.xlsx")
# Print the top few elements of the dataframe as a check
print(df.head())

                        Combination                                  Name  \
0  13:ULS_P_B_S 1 (G + Qi Tch + Qt)  1:Loadcase 1 (G | Permanent Actions)   
1   17:ULS_P_B_S 2 (G + Qi Hb + Qt)  1:Loadcase 1 (G | Permanent Actions)   
2           19:ULS_P_B_S 3 (G + Qt)  1:Loadcase 1 (G | Permanent Actions)   
3           21:ULS_P_B_S 4 (G + Qt)  1:Loadcase 1 (G | Permanent Actions)   
4                23:ULS_P_B_S 5 (G)  1:Loadcase 1 (G | Permanent Actions)   

   Permanent  Variable                                               Name  \
0        1.0      0.35  2:Loadcase 2 (Qi Tch | Traffic Load - characte...   
1        1.0      0.35  3:Loadcase 3 (Qi Hb | Horizontal forces - brak...   
2        1.0      0.35                       12:Loadcase 4 (Qt | Thermal)   
3        1.0      0.35                       12:Loadcase 4 (Qt | Thermal)   
4        1.0      0.35                                               None   

   Permanent  Variable                          Name  Permanent  Variable 